In [1]:
import numpy as np
import pygsti as pig
import matplotlib.pyplot as plt
import random
import time
import pandas
from scipy.linalg import expm, sqrtm
# from qutip import *

from pygsti.modelmembers.operations import LindbladErrorgen as _LinbladErrorgen
from pygsti.modelmembers.operations import EmbeddedErrorgen as _EmbeddedErrorgen
from pygsti.modelmembers.operations import ExpErrorgenOp as _ExpErrorgenOp
from pygsti.modelmembers.operations import ComposedOp as _ComposedOp
import  pygsti.modelmembers as _mdmb
import pygsti.processors as _proc

from pygsti.tools import pdftools as _pdftools

from pygsti.circuits.gstcircuits import create_lsgst_circuits

from pygsti.models.memberdict import OrderedMemberDict as _OrderedMemberDict
from pygsti.baseobjs.label import Label as _Label, CircuitLabel as _CircuitLabel
from pygsti.circuits import Circuit
import pygsti.circuits as _circ
import pygsti.models as _mdl
from pygsti.modelmembers import operations as _op

In [2]:
def random_circuit(depth, proc_spec):
    """
    works for any 1-qubit model pack and the XYZICnot 2-qubit pack
    """
    layers = []
    for i in range(depth):
        gate = random.choice(proc_spec.gate_names)
        if gate == '(idle)':
            pass
        elif gate == 'Gcnot':
            layers.append((gate, 0, 1))
        else:
            layers.append((gate, random.choice(proc_spec.qubit_labels)))
    return Circuit(layers, proc_spec.qubit_labels)

In [3]:
def vector_from_outcomes(outcomes, num_outcomes):
    vecout = np.zeros((num_outcomes))
    for key in outcomes.keys():
        vecout[int(key[0], 2)] = outcomes[key]
    return(vecout)

def matrix_from_jacob(jacob, num_outcomes):
    matout = np.zeros((num_outcomes, len(jacob['0'*int(np.log2(num_outcomes))])))
    for key in jacob.keys():
        matout[int(key[0], 2), :] = np.array(jacob[key])
    return matout

def tensor_from_hessian(hessian, hilbert_dims):
    """
    returns a 3d-array that when dotted into the state returns the jacobian 
    """
    num_params = len(hessian['0'*int(np.log2(hilbert_dims))])
    tensor_out = np.zeros((hilbert_dims, num_params, num_params))
    for key in hessian.keys():
        tensor_out[int(key[0], 2), :, :] = hessian[key]
    return tensor_out

In [4]:
def compare_models(true_model, model1, model2, circuit_list, name1='x', name2='y'):
    distribution1 = []
    distribution2 = []
    for idx, circ in enumerate(circuit_list): 
        true_outcomes = true_model.probabilities(circ)
        outcome_set1 = model1.probabilities(circ)
        outcome_set2 = model2.probabilities(circ)
        dist1 = _pdftools.tvd(true_outcomes, outcome_set1)
        dist2 = _pdftools.tvd(true_outcomes, outcome_set2)
        distribution1.append(dist1)
        distribution2.append(dist2)
        plt.scatter(dist1, dist2)
    plt.xlabel(name1)
    plt.ylabel(name2)
    mx = max([max(distribution1), max(distribution2)])
    plt.plot((0, mx), (0, mx), c='black')
    plt.show()
    return (distribution1, distribution2)

In [5]:
def calculate_jacobs(circ_list, model):
    jacobs = {}
    for circ in circ_list:
        hilbert_dims = 2**circ.width
        jacobs[circ] = matrix_from_jacob(model.sim.dprobs(circ), hilbert_dims)
    return jacobs

In [6]:
from pygsti.processors import CliffordCompilationRules as CCR

def make_rb_param(noise_model, model_pack, length_powers=5, circuits_at_length=10):
    pspec = model_pack.processor_spec()
    
    depths = [2**i for i in range(length_powers)]
    
    compilations = {'absolute': CCR.create_standard(pspec, 'absolute', ('paulis', '1Qcliffords'), verbosity=0),            
                'paulieq': CCR.create_standard(pspec, 'paulieq', ('1Qcliffords', 'allcnots'), verbosity=0)}

    design = pig.protocols.DirectRBDesign(model_pack.processor_spec(), compilations, depths, circuits_at_length, qubit_labels=model_pack.processor_spec().qubit_labels, sampler='edgegrab', 
                                           samplerargs=[0.5], randomizeout=True,
                                           citerations=20)
    pig.io.write_empty_protocol_data(design, 'RB_Data', clobber_ok=True)
    pig.io.fill_in_empty_dataset_with_fake_data(noise_model, 'RB_Data/data/dataset.txt', num_samples=1000)
    data = pig.io.load_data_from_dir('RB_Data')
    protocol = pig.protocols.RB()
    results = protocol.run(data)
    return results.fits['full'].estimates['r'] 

In [7]:
def make_dirichlet_covar(counts, hilbert_dims):
    total_counts = sum([counts[key] for key in counts.keys()])    
    count_vec = np.ones(hilbert_dims)
    for key in counts.keys():
        count_vec[int(key[0], 2)] += counts[key]
    prefactor = 1/( (total_counts + hilbert_dims)**2 * (total_counts + hilbert_dims + 1) )
    meas_covar = prefactor*(
        (total_counts + hilbert_dims)*np.diag(count_vec) - np.outer(count_vec, count_vec)
    )
    return meas_covar

In [34]:
from filterpy.kalman import KalmanFilter

class LinearKalmanFilter(KalmanFilter):
    def __init__(self, x0, P0, hilbert_dims):
        super().__init__(len(x0), hilbert_dims)
        self.x = x0
        self.P = P0
        
        self.F = np.eye(len(x0))
        self.Q = np.zeros((len(x0), len(x0)))
        
    def update_filter(self, circ, count_vec, pvec_model, jacob, stab_noise=None):
        
        total_counts = sum(count_vec)
        
        hilbert_dims = 2**circ.width
        
        prediction = (total_counts*pvec_model + np.ones(hilbert_dims))/(total_counts + hilbert_dims)
        observation = (count_vec + np.ones(hilbert_dims))/(total_counts + hilbert_dims)
        
        self.H = jacob
        shot_noise = (1/total_counts)*make_multinom_covar(observation)
        if stab_noise is not None:
            shot_noise += stab_noise
        self.R = shot_noise
        
        self.update(observation - prediction)

In [84]:
def make_multinom_covar(prob_vec):
    return np.eye(len(prob_vec)) - np.outer(prob_vec, prob_vec)

In [12]:
from pygsti.modelpacks import smq1Q_XYI as _smq1Q_XYI
from pygsti.modelpacks import smq1Q_XYZI as _smq1Q_XYZI
from pygsti.modelpacks import smq1Q_XYI as _smq1Q_XZ
from pygsti.modelpacks import smq2Q_XYZICNOT as _smq2Q_XYZICNOT

In [15]:
# make random circuits and design matrices
N_circs = 500
depth = 64

circ_list = []
for n in range(N_circs):
    circ_list.append(random_circuit(random.choice(range(depth)), MODEL_PACK.processor_spec()))
circ_list = _circ.to_circuits(circ_list) 

jdict = dict()
#hdict = dict()
for idx, circ in enumerate(circ_list):
    if idx%10 == 0:
        print(idx/10, N_circs/10)
    jdict[circ] = matrix_from_jacob(ref_model.sim.dprobs(circ), 2**circ.width)
    #hdict[circ] = tensor_from_hessian(ref_model.sim.hprobs(circ), 2**circ.width)

0.0 50.0
1.0 50.0
2.0 50.0
3.0 50.0
4.0 50.0
5.0 50.0
6.0 50.0
7.0 50.0
8.0 50.0
9.0 50.0
10.0 50.0
11.0 50.0
12.0 50.0
13.0 50.0
14.0 50.0
15.0 50.0
16.0 50.0
17.0 50.0
18.0 50.0
19.0 50.0
20.0 50.0
21.0 50.0
22.0 50.0
23.0 50.0
24.0 50.0
25.0 50.0
26.0 50.0
27.0 50.0
28.0 50.0
29.0 50.0
30.0 50.0
31.0 50.0
32.0 50.0
33.0 50.0
34.0 50.0
35.0 50.0
36.0 50.0
37.0 50.0
38.0 50.0
39.0 50.0
40.0 50.0
41.0 50.0
42.0 50.0
43.0 50.0
44.0 50.0
45.0 50.0
46.0 50.0
47.0 50.0
48.0 50.0
49.0 50.0


# Linear Kalman Filter 

In [76]:
# make a model from a model pack
MODEL_PACK = _smq1Q_XYZI
REF_MODEL = MODEL_PACK.target_model('H+S')
filter_model = MODEL_PACK.target_model('H+S')
noise_model = MODEL_PACK.target_model()
noise_model = noise_model.depolarize(max_op_noise=0.01, max_spam_noise=0.001)
noise_model = noise_model.rotate(max_rotate=0.01)

In [77]:
# make rb param
rb_param = make_rb_param(noise_model, MODEL_PACK)
print(rb_param, np.sqrt(rb_param))

- Sampling 10 circuits at DRB length 1 (1 of 5 depths) with seed 621485
- Sampling 10 circuits at DRB length 2 (2 of 5 depths) with seed 621495
- Sampling 10 circuits at DRB length 4 (3 of 5 depths) with seed 621505
- Sampling 10 circuits at DRB length 8 (4 of 5 depths) with seed 621515
- Sampling 10 circuits at DRB length 16 (5 of 5 depths) with seed 621525
0.002755391098345189 0.052491819346877175


In [78]:
# sample circuits
num_samples = 1000
dataset_random = pig.data.simulate_data(noise_model, circ_list, num_samples=num_samples)
shot_noise = 1/np.sqrt(num_samples)

In [79]:
# setup the filter
from filterpy.kalman import KalmanFilter

hilbert_dims = 2**len(MODEL_PACK.processor_spec().qubit_labels)
num_params = len(filter_model.to_vector())

kf = LinearKalmanFilter(np.zeros(num_params), 2*np.sqrt(rb_param)*np.eye(num_params), hilbert_dims)

In [80]:
# run it
%matplotlib

plotting = True

for idx, circ in enumerate(circ_list): 
    if plotting:
        filter_model.from_vector(kf.x)
        naive_error = _pdftools.tvd(REF_MODEL.probabilities(circ), dataset_random[circ].fractions)
        error = _pdftools.tvd(filter_model.probabilities(circ), dataset_random[circ].fractions)
        plt.scatter(idx, naive_error, c='blue')
        plt.scatter(idx, error, c='red', alpha=0.5)
        
    counts = vector_from_outcomes(data_set[circ].counts, hilbert_dims)
    prob = vector_from_outcomes(REF_MODEL.probabilities(circ), hilbert_dims)
    kf.update_filter(circ, counts, prob, jdict[circ], 1e-6*np.eye(hilbert_dims))
    


    
        
#     if(plotting):
#         plt.scatter(idx, 0.5*np.linalg.norm(kf.y))
#         naive_error = _pdftools.tvd(ref_model.probabilities(circ), data_set[circ].fractions)
#         plt.scatter(idx, naive_error, c='blue', alpha=0.5)
#         error = _pdftools.tvd(filter_model.probabilities(circ), data_set[circ].fractions)
#         plt.scatter(idx, error, c='red', alpha=0.5)

Using matplotlib backend: Qt5Agg


In [81]:
# update the model
filter_model.from_vector(kf.x)

In [82]:
# make random circuits 
plt.figure()
N_circs = 250
depth = 10
new_circ_list = []
for n in range(N_circs):
    new_circ_list.append(random_circuit(depth, MODEL_PACK.processor_spec()))

refTVDs, filterTVDs = compare_models(noise_model, ref_model, filter_model, new_circ_list, 'ref', 'filter')

In [83]:
print(len(refTVDs))

250


In [92]:
same_TVDs = {}
for idx, tvd in enumerate(refTVDs):
    if (refTVDs[idx] - filterTVDs[idx]) < 1e-6:
        same_TVDs[new_circ_list[idx]] = filterTVDs


In [93]:
same_circs = [key for key in same_TVDs.keys()]

In [94]:
print(same_circs[-1])

Qubit 0 ---|Gzpi2|-|Gypi2|-|Gxpi2|-|Gzpi2|-|Gxpi2|-|Gxpi2|-|Gypi2|---



In [95]:
print(len(same_circs), len(new_circ_list))

18 250


In [96]:
plt.figure()
plt.hist(filterTVDs)

(array([72., 55., 26.,  7.,  6., 15., 36., 20., 12.,  1.]),
 array([3.71162902e-05, 3.09882185e-03, 6.16052740e-03, 9.22223296e-03,
        1.22839385e-02, 1.53456441e-02, 1.84073496e-02, 2.14690552e-02,
        2.45307607e-02, 2.75924663e-02, 3.06541719e-02]),
 <BarContainer object of 10 artists>)

In [98]:
for c in same_circs:
    if c in dataset_random.keys():
        print(c, data_set[c], jdict[c])

In [100]:
for c in same_circs:
    print(REF_MODEL.probabilities(c))

OutcomeLabelDict([(('0',), 0.49999999999999933), (('1',), 0.5)])
OutcomeLabelDict([(('0',), 0.49999999999999956), (('1',), 0.49999999999999956)])
OutcomeLabelDict([(('0',), 0.5000000000000002), (('1',), 0.49999999999999917)])
OutcomeLabelDict([(('0',), 0.49999999999999944), (('1',), 0.49999999999999967)])
OutcomeLabelDict([(('0',), 0.49999999999999967), (('1',), 0.49999999999999967)])
OutcomeLabelDict([(('0',), 0.5), (('1',), 0.5000000000000004)])
OutcomeLabelDict([(('0',), 0.4999999999999998), (('1',), 0.4999999999999998)])
OutcomeLabelDict([(('0',), 0.5000000000000002), (('1',), 0.4999999999999994)])
OutcomeLabelDict([(('0',), 0.4999999999999997), (('1',), 0.4999999999999993)])
OutcomeLabelDict([(('0',), 0.5), (('1',), 0.49999999999999956)])
OutcomeLabelDict([(('0',), 0.5), (('1',), 0.4999999999999998)])
OutcomeLabelDict([(('0',), 0.49999999999999944), (('1',), 0.4999999999999999)])
OutcomeLabelDict([(('0',), 0.4999999999999996), (('1',), 0.5000000000000002)])
OutcomeLabelDict([(('0'